In [2]:
# Author Emmanuel Sedicol
import cv2
import imutils
import numpy as np
import import_ipynb
import MAIN_APP as mn
import tensorflow as tf
import matplotlib.pyplot as plt
import POSITION_EXTRACTION as pex

from IPython.display import clear_output
from imutils.video import VideoStream
from imutils.video import FPS
from skimage import measure
from YOLO import yolo

DEBUG = True
model = tf.keras.models.load_model("MODEL")

importing Jupyter notebook from MAIN_APP.ipynb
importing Jupyter notebook from POSITION_EXTRACTION.ipynb


# Ball Detection Test

In [3]:
if DEBUG:
    shot_xy_coordinates = []
    path = "VIDEOS/BALL_DETECTION_1.mov"

    vs = cv2.VideoCapture(path)
    fps = FPS().start()

    while True:
        # grab the current frame
        _,frame = vs.read()

        # break out of the application if we have no more frames to read
        if frame is None:
            break

        # resize frame so to make its constant size througout
        frame = cv2.resize(frame,(600, 400))
        detected_object = frame.copy()
        
        x, y, frame = mn.opencv_ball_detection(frame, True)
        shot_xy_coordinates.append((x,y))
        

        # plot results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
        fig.suptitle('Basketball and Player Detection')
        ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # plot at realtime the shot coordinates
        for x in range(0, len(shot_xy_coordinates)):
            ax2.scatter(shot_xy_coordinates[x][0], shot_xy_coordinates[x][1])

        # Plot Limit equals the size of the frame
        ax2.set_xlim(0, 600)
        ax2.set_ylim(0,400)
        ax2.invert_yaxis()

        plt.show()
        clear_output(wait=True)

    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    vs.release()

KeyboardInterrupt: 

# Player Detection Test
- YOLO Detection
- Background Subtraction Dectection

### YOLO Detection Test

In [6]:
if DEBUG:    
    path = "VIDEOS/BALL_DETECTION_2.mov"

    vs = cv2.VideoCapture(path)
    fps = FPS().start()

    while True:
        # grab the current frame
        _,frame = vs.read()

        # break out of the application if we have no more frames to read
        if frame is None:
            break

        # resize frame so to make its constant size througout
        frame = cv2.resize(frame,(600, 450))
        detected_object = frame.copy()

        # detect person using YOLO
        target = "person"
        yolo.yolo_detection(target, frame)

        # plot results
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        fig.suptitle('Basketball and Player Detection')
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        ax.axis("off")
        plt.show()
        clear_output(wait=True)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    vs.release()

KeyboardInterrupt: 

### Detection using Background Subtraction Method

In [7]:

vs = cv2.VideoCapture('VIDEOS/BALL_DETECTION_1.mov') 
fps = FPS().start()

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

while True:
    # grab the current frame
    _, frame = vs.read()

    if frame is None:
        break
        
    frame = cv2.resize(frame,(600, 400))

    blur = cv2.GaussianBlur(frame, (15, 15),0)
    dilate = cv2.dilate(blur, None, iterations=2)
    fgmask = fgbg.apply(dilate)
    dilate = cv2.dilate(fgmask, kernel)

    image, cnts, heirarchy = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(cnts):
        area = cv2.contourArea(contour)

        if area > 1000:
            x,y,w,h = cv2.boundingRect(contour)

            x_coordinate = int(x + (w/2))
            y_coordinate = int(y + h)

            cv2.rectangle(frame, (x,y), (x+w, y+h), (90,10,255), 2)
            cv2.putText(frame, "PERSON", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1 )
            cv2.circle(frame, (x_coordinate, y_coordinate), 3, (0, 0, 255), 8)


    fig, (ax1) = plt.subplots(figsize=(24, 8))
    ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax1.axis("off")
    plt.show()
    clear_output(wait=True)

    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    vs.release()

[INFO] elasped time: 0.66
[INFO] approx. FPS: 0.00
